In [1]:
###AutoTomb##
##Takes Digital Giza tomb page URL and returns a set of AI (Meshy) generated 3D models corresponding to contemporaneous (i.e., ancient Egyptian) object references mentioned in early 20th century excavation diaries##
###Cook2025 - mncook.net###

In [ ]:
##global config
import os
import logging
import requests
import json
import time
import re
import openai
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import umap.umap_ as umap
from collections import defaultdict

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# API Keys and Endpoints
HARVARD_API_KEY = "..."
MESHY_API_KEY = "..."

OPENAI_API_BASE = "..."
MESHY_API_URL = "..."

# Target Tomb
MAIN_TOMB_URL = "http://giza.fas.harvard.edu/sites/532/full/"

# GPT and Meshy Models
GPT_MODEL_NAME = "gpt-4.1"
TEMPERATURE = 0.0
IMAGE_MODEL = "dall-e-3"

# User input for base directory
BASE_DIR = "..."

# Directory Setup
ARTIFACTS_JSON = os.path.join(BASE_DIR, "artifacts.json")
IMAGES_DIR = os.path.join(BASE_DIR, "Images")
MODELS_DIR = os.path.join(BASE_DIR, "Models")
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Create additional subdirectories under "Models"
GLTF_MODELS_DIR = os.path.join(MODELS_DIR, "gltf")
os.makedirs(GLTF_MODELS_DIR, exist_ok=True)

# Ensure artifacts.json exists
if not os.path.exists(ARTIFACTS_JSON):
    logger.info("No artifacts.json found, creating a new one.")
    with open(ARTIFACTS_JSON, 'w') as json_file:
        json.dump({"artifacts": []}, json_file, indent=2)

logger.info(f"Directories and artifacts.json setup completed at {BASE_DIR}")


In [ ]:
##Scrape diary pages and generate prompts
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# Configuration from global config
PAGE_RANGE = None

# Scraper function
def scrape_diary_transcriptions(main_url, page_range=None):
    logger.info("Initializing Selenium Chrome WebDriver.")
    service = ChromeService()
    driver = webdriver.Chrome(service=service)

    results = []
    try:
        logger.info(f"Navigating to main tomb page: {main_url}")
        driver.get(main_url)
        time.sleep(3)

        tomb_name = driver.find_element(By.CSS_SELECTOR, "body > div.page-header.header-bg-1 > div > header > h1").text
        logger.info(f"Extracted tomb name: {tomb_name}")

        links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/diarypages/'][href*='/full/']")
        diary_links = list({elem.get_attribute("href") for elem in links})
        logger.info(f"Found {len(diary_links)} unique diary page links.")

        start, end = page_range if page_range else (0, len(diary_links))
        diary_links = diary_links[start:end]
        logger.info(f"Processing diary page range: {start} to {end}")

        for link in diary_links:
            logger.info(f"Processing diary link: {link}")
            driver.get(link)
            time.sleep(2)

            excavation_date, transcription_text = "", ""
            try:
                div = driver.find_element(By.CSS_SELECTOR, "div.item__overview.text-alt")
                paragraphs = div.find_elements(By.TAG_NAME, "p")
                transcription_text = "\n".join(p.text for p in paragraphs)

                lines = transcription_text.split("\n")
                date_match = re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),.*?\d{4}", lines[1])
                if date_match:
                    excavation_date = date_match.group().strip()
                    logger.info(f"Extracted excavation date: {excavation_date}")
                else:
                    logger.warning(f"No excavation date found in {link}")

            except Exception as e:
                logger.warning(f"Issue parsing transcription div or date for {link}: {e}")

            results.append({"url": link, "text": transcription_text, "excavation_date": excavation_date, "tomb_id": tomb_name})
    finally:
        driver.quit()

    return results

# GPT Parsing Function
def parse_diary_text_for_objects(diary_text):
    headers = {"Content-Type": "application/json", "api-key": HARVARD_API_KEY}
    url = "https://go.apis.huit.harvard.edu/ais-openai-direct/v1/chat/completions"

    payload = {
        "model": GPT_MODEL_NAME,
        "temperature": TEMPERATURE,
        "messages": [{
            "role": "system",
            "content": (
                "Extract all contemporaneous ancient physical objects mentioned in the excavation diary. "
                "For each artifact, output detailed metadata in structured JSON suitable for 3D model generation, including visual attributes."
                "Include fragmentary, incomplete, and minor objects. Format:"
                "{artifact_id, mention, prompt, context, tomb_id, excavation_location(area, coordinates), artifact_attributes(type, material, condition, color, inscriptions_present, orientation, period)}"
            )
        }, {"role": "user", "content": diary_text}]
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        logger.error(f"GPT API error ({response.status_code}): {response.text}")
        return []

    content = response.json()["choices"][0]["message"]["content"]
    content = re.sub(r"```(?:json)?", "", content).replace("```", "").strip()

    try:
        artifacts = json.loads(content)
        return artifacts if isinstance(artifacts, list) else artifacts.get("artifacts", [])
    except json.JSONDecodeError as e:
        logger.error(f"JSON Parsing Error: {e}")
        return []

# Main execution
def main():
    start_time = time.time()
    pages = scrape_diary_transcriptions(MAIN_TOMB_URL, PAGE_RANGE)

    artifacts = []
    for page in pages:
        if not page["text"].strip():
            continue

        parsed_artifacts = parse_diary_text_for_objects(page["text"])
        for artifact in parsed_artifacts:
            artifact.update({"source_url": page["url"], "excavation_date": page["excavation_date"], "tomb_id": page["tomb_id"]})
        artifacts.extend(parsed_artifacts)

    if os.path.exists(ARTIFACTS_JSON):
        with open(ARTIFACTS_JSON, "r") as f:
            existing_data = json.load(f).get("artifacts", [])
        artifacts = existing_data + artifacts

    with open(ARTIFACTS_JSON, "w", encoding="utf-8") as f:
        json.dump({"artifacts": artifacts}, f, ensure_ascii=False, indent=2)

    total_time = time.time() - start_time
    logger.info(f"Scraping and parsing completed in {total_time:.2f} seconds. Data saved to {ARTIFACTS_JSON}")

if __name__ == "__main__":
    main()


In [ ]:
##Generate images from diary prompt list
def generate_gpt_image(prompt, artifact_id):
    contextual_prefix = ("Highly detailed, photorealistic rendering in authentic Old Kingdom Egyptian "
                     "style, appropriate for archaeological reconstruction. ")

    # Updated prompt in the image payload
    image_payload = {
        "model": IMAGE_MODEL,
        "prompt": f"{contextual_prefix} {prompt}",
        "size": "1024x1024",
        "quality": "standard",
        "n": 1
    }

    headers = {"Content-Type": "application/json", "api-key": HARVARD_API_KEY}
    image_url = f"{OPENAI_API_BASE}/images/generations"

    logger.info(f"Generating image for artifact ID {artifact_id}.")
    response = requests.post(image_url, headers=headers, json=image_payload)
    if response.status_code != 200:
        logger.error(f"Image generation error for artifact ID {artifact_id}: {response.text}")
        return None

    image_link = response.json()['data'][0]['url']
    image_response = requests.get(image_link)

    if image_response.status_code == 200:
        image_path = os.path.join(IMAGES_DIR, f"{artifact_id}.png")
        with open(image_path, 'wb') as img_file:
            img_file.write(image_response.content)
        logger.info(f"Image saved: {image_path}")
        return image_path
    else:
        logger.error(f"Failed to download generated image for artifact ID {artifact_id}.")
        return None

# Main Workflow
def main():
    start_time = time.time()

    with open(ARTIFACTS_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    artifacts = data.get("artifacts", [])

    for artifact in artifacts:
        artifact_id = artifact["artifact_id"]
        prompt = artifact["prompt"]
        image_path = generate_gpt_image(prompt, artifact_id)
        artifact["image_path"] = image_path if image_path else ""

    with open(ARTIFACTS_JSON, "w", encoding="utf-8") as f:
        json.dump({"artifacts": artifacts}, f, ensure_ascii=False, indent=2)

    total_time = time.time() - start_time
    logger.info(f"Completed in {total_time:.2f} seconds. Updated JSON saved to {ARTIFACTS_JSON}")

if __name__ == "__main__":
    main()


In [ ]:
##Meshy 3D model generation script (GLB, textures, 10,000 polycount)

# Constants
MESHY_STATUS_ENDPOINT = "https://api.meshy.ai/openapi/v1/image-to-3d/{task_id}"
HEADERS = {
    "Authorization": f"Bearer {MESHY_API_KEY}",
    "Content-Type": "application/json"
}

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# Load artifact JSON
with open(ARTIFACTS_JSON, 'r') as f:
    data = json.load(f)
artifacts = data.get("artifacts", [])

# Existing models check for partial execution
existing_models = {os.path.splitext(f)[0] for f in os.listdir(MODELS_DIR) if f.endswith(".glb")}

def encode_image_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/png;base64,{encoded}"

def submit_to_meshy(image_base64):
    payload = {
        "image_url": image_base64,
        "enable_pbr": True,
        "should_remesh": True,
        "should_texture": True,
        "ai_model": "meshy-4",
        "target_polycount": 10000
    }
    response = requests.post(MESHY_API_URL, headers=HEADERS, json=payload)
    if response.status_code in [200, 202]:
        task_id = response.json().get("result")
        logger.info(f"Task submitted successfully: {task_id}")
        return task_id
    else:
        logger.error(f"Failed to submit task: {response.status_code}, {response.text}")
        return None

def wait_for_task_completion(task_id, timeout=600, interval=15):
    start_time = time.time()
    while True:
        response = requests.get(MESHY_STATUS_ENDPOINT.format(task_id=task_id), headers=HEADERS)
        if response.status_code != 200:
            logger.error(f"Error polling task {task_id}: {response.status_code}, {response.text}")
            return None
        
        result = response.json()
        status = result.get("status")
        logger.info(f"Task {task_id} status: {status}")

        if status == "SUCCEEDED":
            return result
        elif status in ["FAILED", "CANCELED"]:
            logger.error(f"Task {task_id} {status.lower()}.")
            return None

        elapsed = time.time() - start_time
        if elapsed > timeout:
            logger.error(f"Task {task_id} timeout after {timeout} seconds.")
            return None
        
        time.sleep(interval)

def download_file(url, output_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        logger.info(f"Downloaded file: {output_path}")
    else:
        logger.error(f"Failed to download file: {response.status_code}, {response.text}")

# Main loop
logger.info("Starting Meshy 3D GLB model generation and download...")
start_time = time.time()

for artifact in artifacts:
    artifact_id = artifact["artifact_id"]

    if artifact_id in existing_models:
        logger.info(f"Model already exists for artifact {artifact_id}, skipping.")
        artifact["model_path"] = os.path.join(MODELS_DIR, f"{artifact_id}.glb")
        continue

    image_path = artifact.get("image_path")
    if not image_path or not os.path.exists(image_path):
        logger.warning(f"No valid image for artifact {artifact_id}, skipping.")
        continue

    logger.info(f"Processing artifact {artifact_id}...")

    image_base64 = encode_image_base64(image_path)
    task_id = submit_to_meshy(image_base64)
    if not task_id:
        continue

    task_result = wait_for_task_completion(task_id)
    if not task_result:
        continue

    model_urls = task_result.get("model_urls", {})
    glb_url = model_urls.get("glb")
    if not glb_url:
        logger.error(f"No GLB model URL found for artifact {artifact_id}.")
        continue

    glb_file_path = os.path.join(MODELS_DIR, f"{artifact_id}.glb")
    download_file(glb_url, glb_file_path)

    texture_urls = task_result.get("texture_urls", [{}])[0]
    local_texture_paths = {}
    for tex_type, tex_url in texture_urls.items():
        tex_path = os.path.join(MODELS_DIR, f"{artifact_id}_{tex_type}.png")
        download_file(tex_url, tex_path)
        local_texture_paths[tex_type] = tex_path

    artifact["model_path"] = glb_file_path
    artifact["textures"] = local_texture_paths

# Update JSON
with open(ARTIFACTS_JSON, 'w') as f:
    json.dump(data, f, indent=2)

end_time = time.time()
total_time = end_time - start_time
logger.info(f"All models processed. Total execution time: {total_time:.2f} seconds.")


In [ ]:
##Embeddings, reduction, and model placement coordinates
# Configuration
HARVARD_API_URL = "..."
MODEL = "text-embedding-3-large"
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

# Load artifacts
with open(ARTIFACTS_JSON, 'r') as f:
    data = json.load(f)
artifacts = data.get("artifacts", [])

# Update model paths (GLB) if they exist
existing_models = {f.replace('.glb', ''): os.path.join(MODELS_DIR, f)
                   for f in os.listdir(MODELS_DIR) if f.endswith('.glb')}

for artifact in artifacts:
    artifact_id = artifact['artifact_id']
    if artifact_id in existing_models:
        artifact['model_path'] = existing_models[artifact_id]

# Filter valid artifacts
valid_artifacts = [a for a in artifacts if a.get('model_path') and a.get('mention') and a.get('prompt')]
if not valid_artifacts:
    logging.error("No valid artifacts with model paths found.")
    exit()

# Embedding function
def get_embedding(text):
    headers = {"Content-Type": "application/json", "api-key": HARVARD_API_KEY}
    data = {"model": MODEL, "input": text}
    response = requests.post(HARVARD_API_URL, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["data"][0]["embedding"]

# Generate embeddings
logging.info(f"Generating embeddings for {len(valid_artifacts)} artifacts...")
embeddings = [get_embedding(f"{a['mention']} {a['prompt']}") for a in valid_artifacts]

# UMAP 3D reduction
logging.info("Reducing embeddings to 3D space...")
reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, n_components=3, random_state=42)
umap_coords = reducer.fit_transform(embeddings)

# Normalize to [0, 1]
coords_min, coords_max = np.min(umap_coords, axis=0), np.max(umap_coords, axis=0)
normalized_coords = (umap_coords - coords_min) / (coords_max - coords_min)

# Spherical projection
logging.info("Converting to spherical layout...")
azimuths = normalized_coords[:, 0] * 2 * np.pi
inclinations = normalized_coords[:, 1] * np.pi
radii = 5 + 5 * normalized_coords[:, 2]

x_coords = radii * np.sin(inclinations) * np.cos(azimuths)
y_coords = radii * np.sin(inclinations) * np.sin(azimuths)
z_coords = radii * np.cos(inclinations)

# Select type specimens and frequency scaling
type_frequency = defaultdict(int)
type_to_best_artifact = {}

for artifact in valid_artifacts:
    artifact_type = artifact["artifact_attributes"]["type"]
    type_frequency[artifact_type] += 1
    current = type_to_best_artifact.get(artifact_type, {"desc_len": -1})
    length = len(artifact['mention']) + len(artifact['prompt'])
    if length > current["desc_len"]:
        type_to_best_artifact[artifact_type] = {"artifact": artifact, "desc_len": length}

# Assign position and scale
max_freq = max(type_frequency.values())

for artifact, x, y, z in zip(valid_artifacts, x_coords, y_coords, z_coords):
    artifact_type = artifact["artifact_attributes"]["type"]
    rep = type_to_best_artifact[artifact_type]["artifact"]

    artifact["vr_position"] = {"x": float(x), "y": float(y), "z": float(z)}
    artifact["type_specimen"] = artifact == rep
    if artifact["type_specimen"]:
        freq = type_frequency[artifact_type]
        artifact["vr_scale"] = round(min(1.0 + np.log(freq), 5.0), 2)
    else:
        artifact["vr_scale"] = 0.0

# Add explanatory key
data["visual_dimensions_key"] = {
    "x": "Azimuthal angle (semantic direction)",
    "y": "Elevation angle (semantic height)",
    "z": "Radial distance from center (semantic prominence)",
    "vr_scale": "Frequency-scaled size of type specimen",
    "type_specimen": "True if this object represents its artifact type"
}

# Save updated JSON
with open(ARTIFACTS_JSON, 'w') as f:
    json.dump(data, f, indent=2)

logging.info("Artifacts JSON updated with spherical layout, representative scaling, and model paths.")


In [ ]:
#Cook 2025